<a href="https://colab.research.google.com/github/raptor0323/csd2003/blob/Lab_submissions/Pyspark_classification_202318017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing pyspark

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=f99a7866e2e8689b231966b57de2493cbd36db1c122ead5ba119e757b35cdfef
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Creating a spark session

In [ ]:
from pyspark.sql import SparkSession


In [ ]:
spark= SparkSession.builder.appName("Online News Popularity ").getOrCreate()

# Reading a file and storing in a data variable

In [ ]:
data= spark.read.csv("/content/OnlineNewsPopularity.csv",header = True,inferSchema=True)

# Familarizing the data

In [ ]:
data.printSchema()

root
 |-- url: string (nullable = true)
 |--  timedelta: double (nullable = true)
 |--  n_tokens_title: double (nullable = true)
 |--  n_tokens_content: double (nullable = true)
 |--  n_unique_tokens: double (nullable = true)
 |--  n_non_stop_words: double (nullable = true)
 |--  n_non_stop_unique_tokens: double (nullable = true)
 |--  num_hrefs: double (nullable = true)
 |--  num_self_hrefs: double (nullable = true)
 |--  num_imgs: double (nullable = true)
 |--  num_videos: double (nullable = true)
 |--  average_token_length: double (nullable = true)
 |--  num_keywords: double (nullable = true)
 |--  data_channel_is_lifestyle: double (nullable = true)
 |--  data_channel_is_entertainment: double (nullable = true)
 |--  data_channel_is_bus: double (nullable = true)
 |--  data_channel_is_socmed: double (nullable = true)
 |--  data_channel_is_tech: double (nullable = true)
 |--  data_channel_is_world: double (nullable = true)
 |--  kw_min_min: double (nullable = true)
 |--  kw_max_min: do

In [ ]:
data.show()

+--------------------+----------+---------------+-----------------+----------------+-----------------+-------------------------+----------+---------------+---------+-----------+---------------------+-------------+--------------------------+------------------------------+--------------------+-----------------------+---------------------+----------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------------+--------------------------+---------------------------+------------------+-------------------+---------------------+--------------------+------------------+--------------------+------------------+-----------+---------------+---------------+---------------+---------------+---------------+--------------------+--------------------------+---------------------------+---------------------------+--------------------+--------------------+----------------------+----------------------+----------------------+-

In [ ]:
data.describe().show()

+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+-------------------------+------------------+------------------+-----------------+------------------+---------------------+------------------+--------------------------+------------------------------+--------------------+-----------------------+---------------------+----------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------------+--------------------------+---------------------------+-------------------+-------------------+---------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------------------+---------------------------

# Data Preprocessing
  ● Perform data cleaning, including dealing with missing values and any irrelevant features.
  ● Since the dataset contains URL data as non-predictive, ensure this is excluded from your analysis.
  ● Convert the dataset's target variable (shares) into a binary outcome for classification (popular vs. not
  popular) based on a threshold you define (e.g., median value).

 Here on operations will be performed on copy of original dataset.

In [ ]:
data2=data.alias('data2')

In [ ]:
data2.show()

+--------------------+----------+---------------+-----------------+----------------+-----------------+-------------------------+----------+---------------+---------+-----------+---------------------+-------------+--------------------------+------------------------------+--------------------+-----------------------+---------------------+----------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------------+--------------------------+---------------------------+------------------+-------------------+---------------------+--------------------+------------------+--------------------+------------------+-----------+---------------+---------------+---------------+---------------+---------------+--------------------+--------------------------+---------------------------+---------------------------+--------------------+--------------------+----------------------+----------------------+----------------------+-

Drop all the null values

In [ ]:
data2.na.drop()

DataFrame[url: string,  timedelta: double,  n_tokens_title: double,  n_tokens_content: double,  n_unique_tokens: double,  n_non_stop_words: double,  n_non_stop_unique_tokens: double,  num_hrefs: double,  num_self_hrefs: double,  num_imgs: double,  num_videos: double,  average_token_length: double,  num_keywords: double,  data_channel_is_lifestyle: double,  data_channel_is_entertainment: double,  data_channel_is_bus: double,  data_channel_is_socmed: double,  data_channel_is_tech: double,  data_channel_is_world: double,  kw_min_min: double,  kw_max_min: double,  kw_avg_min: double,  kw_min_max: double,  kw_max_max: double,  kw_avg_max: double,  kw_min_avg: double,  kw_max_avg: double,  kw_avg_avg: double,  self_reference_min_shares: double,  self_reference_max_shares: double,  self_reference_avg_sharess: double,  weekday_is_monday: double,  weekday_is_tuesday: double,  weekday_is_wednesday: double,  weekday_is_thursday: double,  weekday_is_friday: double,  weekday_is_saturday: double,  w

Dropping column URL which is less relevant here.

In [ ]:
data2=data2.drop("url")

Converting dataset target variables to binary format with threshhold of median.

In [ ]:
median_data=data2.approxQuantile(' shares',[0.5],0.05)[0]


In [ ]:
# Deleting blank spaces
data2 = data2.withColumnRenamed(' shares', 'shares')


In [ ]:
from pyspark.sql.functions import expr
data2=data2.withColumn('popular',expr(f"case WHEN shares >= {median_data} THEN 1 ELSE 0 END"))

In [ ]:
# data2.show()
data2.printSchema()

root
 |--  timedelta: double (nullable = true)
 |--  n_tokens_title: double (nullable = true)
 |--  n_tokens_content: double (nullable = true)
 |--  n_unique_tokens: double (nullable = true)
 |--  n_non_stop_words: double (nullable = true)
 |--  n_non_stop_unique_tokens: double (nullable = true)
 |--  num_hrefs: double (nullable = true)
 |--  num_self_hrefs: double (nullable = true)
 |--  num_imgs: double (nullable = true)
 |--  num_videos: double (nullable = true)
 |--  average_token_length: double (nullable = true)
 |--  num_keywords: double (nullable = true)
 |--  data_channel_is_lifestyle: double (nullable = true)
 |--  data_channel_is_entertainment: double (nullable = true)
 |--  data_channel_is_bus: double (nullable = true)
 |--  data_channel_is_socmed: double (nullable = true)
 |--  data_channel_is_tech: double (nullable = true)
 |--  data_channel_is_world: double (nullable = true)
 |--  kw_min_min: double (nullable = true)
 |--  kw_max_min: double (nullable = true)
 |--  kw_avg

 # 3. Exploratory Data Analysis (EDA)

In [ ]:
data2.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+-------------------------+------------------+------------------+-----------------+------------------+---------------------+------------------+--------------------------+------------------------------+--------------------+-----------------------+---------------------+----------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------------+--------------------------+---------------------------+-------------------+-------------------+---------------------+--------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+--------------------+--------------------------+---------------------------+--------------------

In [ ]:
data2.groupby('popular').count().show()

+-------+-----+
|popular|count|
+-------+-----+
|      1|21154|
|      0|18490|
+-------+-----+



In [ ]:
t=[' n_tokens_content',' num_hrefs',' num_imgs',' global_rate_positive_words',' title_subjectivity']

for i in t:
    data2.groupby('popular').avg(i).show()

+-------+----------------------+
|popular|avg( n_tokens_content)|
+-------+----------------------+
|      1|     564.7633544483313|
|      0|      525.636884802596|
+-------+----------------------+

+-------+------------------+
|popular|   avg( num_hrefs)|
+-------+------------------+
|      1|11.839604802874161|
|      0| 9.790048674959438|
+-------+------------------+

+-------+------------------+
|popular|    avg( num_imgs)|
+-------+------------------+
|      1| 5.066134064479531|
|      0|3.9469442942130883|
+-------+------------------+

+-------+--------------------------------+
|popular|avg( global_rate_positive_words)|
+-------+--------------------------------+
|      1|             0.04061741856300708|
|      0|              0.0384892378328054|
+-------+--------------------------------+

+-------+------------------------+
|popular|avg( title_subjectivity)|
+-------+------------------------+
|      1|      0.2921790891267054|
|      0|      0.2711115835376853|
+-------+--------

Conclusion:-                           

n_tokens_content - we found higher average of n_tokens_content for popular "1", we can conclude that more the number of words the articles will be popular.

num_href- we found higher average of num_href for popular "1" , we can conclude that more the number of hyperlinks the article will be popular.

num_imgs - We found higher average of num_imgs for popular "1", we can conclude that more the number of images the article will be popular

global_rate_positive_words- We found higher average of global_rate_positive_words for popular "1", we can conclude that more the of positivity percentage in article tend for popular

title_subjectivity- We found higher avarage of
title_subjectivity for popular "1" , we can conclude that more precise the title or attractive is more tend to popular




In [ ]:
from pyspark.sql.functions import corr

for j in t:
    print(f"Correlation between 'popular' and '{j}':")
    data2.select(corr('popular', j)).show()

Correlation between 'popular' and ' n_tokens_content':
+--------------------------------+
|corr(popular,  n_tokens_content)|
+--------------------------------+
|            0.041432711278559096|
+--------------------------------+

Correlation between 'popular' and ' num_hrefs':
+-------------------------+
|corr(popular,  num_hrefs)|
+-------------------------+
|      0.09022882461792459|
+-------------------------+

Correlation between 'popular' and ' num_imgs':
+------------------------+
|corr(popular,  num_imgs)|
+------------------------+
|     0.06719315445303688|
+------------------------+

Correlation between 'popular' and ' global_rate_positive_words':
+------------------------------------------+
|corr(popular,  global_rate_positive_words)|
+------------------------------------------+
|                       0.06091682751840865|
+------------------------------------------+

Correlation between 'popular' and ' title_subjectivity':
+----------------------------------+
|corr(popula

# 4. Feature Engineering.                      

In [ ]:
data2 = data2.withColumnRenamed(' n_tokens_content', 'n_tokens_content')

In [ ]:
from pyspark.sql.functions import when

data2= data2.withColumn('length_claasify', when(data2.n_tokens_content<500,'short').
                                           when((data2.n_tokens_content>=500)&(data2.n_tokens_content<1000),'medium').
                                           otherwise('long'))

In [ ]:
data2 = data2.withColumnRenamed(' num_imgs', 'num_imgs')
data2 = data2.withColumnRenamed(' num_hrefs', 'num_hrefs')

In [ ]:
from pyspark.sql.functions import length

#
data2 = data2.withColumn("img_text_ratio", data2.num_imgs / data2.n_tokens_content)
data2 = data2.withColumn("href_density", data2.num_hrefs / data2.n_tokens_content)



In [ ]:
from pyspark.ml.feature import VectorAssembler

features = ['n_tokens_content', 'num_hrefs', 'num_imgs', 'img_text_ratio', 'href_density']

assembler = VectorAssembler(inputCols=features, outputCol='features_vector')

data2 = assembler.transform(data2)


In [ ]:
data2.show()

+----------+---------------+----------------+----------------+-----------------+-------------------------+---------+---------------+--------+-----------+---------------------+-------------+--------------------------+------------------------------+--------------------+-----------------------+---------------------+----------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------------------------+--------------------------+---------------------------+------------------+-------------------+---------------------+--------------------+------------------+--------------------+------------------+-----------+---------------+---------------+---------------+---------------+---------------+--------------------+--------------------------+---------------------------+---------------------------+--------------------+--------------------+----------------------+----------------------+----------------------+----------------------+--

# 5. Model Building  & 6. Model Evaluation                          

In [ ]:
# Spliting the dataset into training and testing sets
train_data, test_data = data2.randomSplit([0.8, 0.2], seed=42)


In [ ]:
data2 = data2.withColumnRenamed(' shares', 'shares')
data2 = data2.withColumnRenamed(' min_positive_polarity', 'min_positive_polarity')
data2 = data2.withColumnRenamed(' max_positive_polarity', 'max_positive_polarity')
data2 = data2.withColumnRenamed(' title_subjectivity', 'title_subjectivity')
# data2 = data2.withColumnRenamed(' shares', 'shares')
# data2 = data2.withColumnRenamed(' shares', 'shares')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

selected_columns = ['min_positive_polarity', 'max_positive_polarity', 'title_subjectivity', 'shares', 'popular']

data_selected = data2.select(selected_columns).na.drop()

assembler = VectorAssembler(inputCols=['min_positive_polarity', 'max_positive_polarity', 'title_subjectivity', 'shares'],
                            outputCol='features')

# Logistic Regression model
lr = LogisticRegression(labelCol='popular', featuresCol='features')

# Pipeline combining assembler and logistic regression
pipeline = Pipeline(stages=[assembler, lr])

# Splitting the data into training and testing sets
train_data, test_data = data_selected.randomSplit([0.8, 0.2], seed=42)

# Fitting the pipeline on the training data
model = pipeline.fit(train_data)

# Making predictions on the test data
predictions = model.transform(test_data)

# Evaluating the model
evaluator = BinaryClassificationEvaluator(labelCol='popular')
auc = evaluator.evaluate(predictions)

print("Accuracy", auc)


Accuracy 0.9999993442336563


Accuracy of the logistic regression is near to one means ' In binary classification, indicating its ability to accurately distinguish between the classes with minimal errors'.

In [ ]:
from pyspark.sql.functions import expr

tp = predictions.filter((predictions["prediction"] == 1) & (predictions["popular"] == 1)).count()
fp = predictions.filter((predictions["prediction"] == 1) & (predictions["popular"] == 0)).count()
tn = predictions.filter((predictions["prediction"] == 0) & (predictions["popular"] == 0)).count()
fn = predictions.filter((predictions["prediction"] == 0) & (predictions["popular"] == 1)).count()

print("Confusion Matrix:")
print("True Positives:", tp)
print("False Positives:", fp)
print("True Negatives:", tn)
print("False Negatives:", fn)


Confusion Matrix:
True Positives: 4154
False Positives: 0
True Negatives: 3671
False Negatives: 0
